In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [3]:
model = ChatOpenAI(streaming=True)
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = model.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [5]:
app.invoke(
    {"messages": [HumanMessage(content="Translate to French: I love programming.")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='What did I just ask you?', additional_kwargs={}, response_metadata={}, id='b2eb2942-e1fb-4010-94b1-9246e6beacab'),
  AIMessage(content='You just asked me, "What did I just ask you?"', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-1223593e-69c8-4652-a1e8-fd01b723665e-0'),
  HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='1c3f2dae-909f-42bb-ad4f-f15b516fdbbd'),
  AIMessage(content='In French, "I love programming" is "J\'adore la programmation."', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-a1d1a7d6-06f5-4444-8416-8934f2b09eee-0')]}

In [7]:
result = app.invoke(
    {"messages": [HumanMessage(content="What did I just ask you?")]},
    config={"configurable": {"thread_id": "1"}},
)

In [8]:
result["messages"][-1].content

'You just asked me, "What did I just ask you?"'

In [6]:
user_input = "Tell me a 300 word story about a dragon and a princess."
for event in app.stream(
    {"messages": [HumanMessage(content=user_input)]},
    config={"configurable": {"thread_id": "1"}},
    ):
    for value in event.values():
        # print("Assistant:", value["messages"][-1].content)
        print(value)

{'messages': AIMessage(content="In the enchanted land of Arindale, a majestic dragon named Aurelia soared through the skies with grace and power. The people of Arindale revered Aurelia, believing her to be a protector of their kingdom. Meanwhile, Princess Elara, known for her compassion and wisdom, spent her days exploring the lush forests and tending to the needs of her people.\n\nOne fateful day, a dark sorcerer descended upon Arindale, casting a shadow of fear and despair over the land. The sorcerer unleashed his dark magic, causing chaos and destruction wherever he went. Princess Elara knew that she had to act quickly to save her kingdom, so she sought the help of Aurelia, the mighty dragon.\n\nAurelia, sensing the princess's distress, descended from the skies and landed before Princess Elara. The princess explained the dire situation to Aurelia, and without hesitation, the dragon pledged to aid her in defeating the sorcerer and restoring peace to Arindale.\n\nTogether, Princess El

In [9]:
user_input = "Tell me a 300 word story about a dragon and a princess."
async for event in app.astream_events({
    "messages": [HumanMessage(content=user_input)]}, 
    config={"configurable": {"thread_id": "1"}},
    version="v1", ):
    kind = event["event"]
    if event['event'] == "on_chat_model_stream":
        # print(event)
        final_event = event
    # print(f"{kind}: {event['name']}")

In [10]:
final_event

{'event': 'on_chat_model_stream',
 'name': 'ChatOpenAI',
 'run_id': '9fefa6c0-116e-4518-a700-f251673536ad',
 'tags': ['seq:step:1'],
 'metadata': {'thread_id': '1',
  'langgraph_step': 16,
  'langgraph_node': 'model',
  'langgraph_triggers': ['start:model'],
  'langgraph_path': ('__pregel_pull', 'model'),
  'langgraph_checkpoint_ns': 'model:73a24472-d3b6-08e7-3116-86ab17e60b36',
  'checkpoint_ns': 'model:73a24472-d3b6-08e7-3116-86ab17e60b36',
  'ls_provider': 'openai',
  'ls_model_name': 'gpt-3.5-turbo',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7},
 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-9fefa6c0-116e-4518-a700-f251673536ad')},
 'parent_ids': []}

In [11]:
user_input = "Tell me a 300 word story about a dragon and a princess."
async for event in app.astream_events({
    "messages": [HumanMessage(content=user_input)]}, 
    config={"configurable": {"thread_id": "1"}},
    version="v1", ):
    kind = event["event"]
    if event['event'] == "on_chat_model_stream":
        print(event['data']['chunk'].content)


In
 the
 kingdom
 of
 Ast
oria
,
 there
 lived
 a
 wise
 dragon
 named
 Orion
 and
 a
 kind
 princess
 named
 Is
abella
.
 Orion
 was
 known
 for
 his
 shimmer
ing
 em
erald
 scales
 and
 his
 ability
 to
 control
 the
 winds
,
 guiding
 storms
 and
 bringing
 rain
 to
 the
 parch
ed
 lands
.
 Princess
 Is
abella
,
 admired
 for
 her
 gentle
 heart
 and
 unw
av
ering
 compassion
,
 spent
 her
 days
 t
ending
 to
 the
 people
 of
 Ast
oria
 and
 exploring
 the
 lush
 forests
 that
 surrounded
 the
 kingdom
.


One
 day
,
 a
 dark
 sor
cerer
 cast
 a
 powerful
 spell
 that
 envelop
ed
 Ast
oria
 in
 an
 eternal
 winter
,
 threatening
 to
 freeze
 the
 kingdom
 and
 its
 inhabitants
 forever
.
 Des
perate
 to
 save
 her
 people
,
 Princess
 Is
abella
 sought
 out
 Orion
,
 believing
 that
 his
 ancient
 magic
 could
 help
 break
 the
 curse
.
 Orion
,
 once
 feared
 by
 the
 kingdom
,
 was
 moved
 by
 Is
abella
's
 courage
 and
 agreed
 to
 join
 forces
 with
 her
.


Together
,
 Is
abel